In [1]:
!python -V

Python 3.9.16


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [6]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/mnt/c/Users/mrozt/Documents/DataTalksClub/mlops-zoomcamp-course/02-experiment-tracking/mlruns/1', creation_time=1684888611882, experiment_id='1', last_update_time=1684888611882, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [7]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [8]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [9]:
len(df_train), len(df_val)

(73908, 61921)

In [10]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [11]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [12]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [13]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.75871520559622

In [14]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [17]:
with mlflow.start_run():

    mlflow.set_tag("developer", "michal")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")

    alpha = 0.005
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [18]:
import xgboost as xgb

In [19]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [20]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [22]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [23]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[23:51:26] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:17.34773                          
[1]	validation-rmse:14.42648                          
[2]	validation-rmse:12.23876                          
[3]	validation-rmse:10.62590                          
[4]	validation-rmse:9.44839                           
[5]	validation-rmse:8.60075                           
[6]	validation-rmse:7.99912                           
[7]	validation-rmse:7.57719                           
[8]	validation-rmse:7.27765                           
[9]	validation-rmse:7.06531                           
[10]	validation-rmse:6.91420                          
[11]	validation-rmse:6.80572                          
[12]	validation-rmse:6.72448                          
[13]	validation-rmse:6.66430                          
[14]	validation-rmse:6.61954                          
[15]	validation-rmse:6.58661                          
[

In [29]:
mlflow.xgboost.autolog(disable=True)

In [37]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.21129959481682314,
        'max_depth': 21,
        'min_child_weight': 1.380060245619262,
        'objective': 'reg:linear',
        'reg_alpha': 0.01822597364200607,
        'reg_lambda': 0.21873705441384753,
        'seed': 42
    }
    mlflow.set_tag("model", "xgboost")
    mlflow.set_tag("developer", "michal")
    mlflow.set_tag("test", "test_of_autolog")
    mlflow.log_params(best_params)
    mlflow.xgboost.autolog(log_models=False) # logs the model a second time, don't need it if using log_model, set log_model to False

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[17:14:57] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:17.45193
[1]	validation-rmse:14.58888
[2]	validation-rmse:12.42402
[3]	validation-rmse:10.80897
[4]	validation-rmse:9.61893
[5]	validation-rmse:8.75631
[6]	validation-rmse:8.13788
[7]	validation-rmse:7.69726
[8]	validation-rmse:7.38052
[9]	validation-rmse:7.15670
[10]	validation-rmse:6.99496
[11]	validation-rmse:6.87848
[12]	validation-rmse:6.79244
[13]	validation-rmse:6.72843
[14]	validation-rmse:6.68147
[15]	validation-rmse:6.64471
[16]	validation-rmse:6.61821
[17]	validation-rmse:6.59851
[18]	validation-rmse:6.58110
[19]	validation-rmse:6.56943
[20]	validation-rmse:6.55899
[21]	validation-rmse:6.55181
[22]	validation-rmse:6.54605
[23]	validation-rmse:6.54106
[24]	validation-rmse:6.53702
[25]	validation-rmse:6.53358
[26]	validation-rmse:6.52986
[27]	validation-rmse:6.52637
[28]	validation-rmse:6.52347
[29]	validation-rmse:6.52166
[30]	validation-r

In [46]:
logged_model = 'runs:/d0b1526fbb3c4161b2ae442c0f246bfe/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# # Predict on a Pandas DataFrame.
# loaded_model.predict(pd.DataFrame(data))

2023/05/26 16:23:40 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.3.2, required: mlflow==2.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


[16:23:41] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


In [47]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: d0b1526fbb3c4161b2ae442c0f246bfe

In [40]:
xgboost_model = mlflow.xgboost.load_model(logged_model)

[17:19:06] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


In [41]:
xgboost_model

In [52]:
import os
os.path.getsize('/mnt/c/Users/mrozt/Documents/DataTalksClub/mlops-zoomcamp-course/02-experiment-tracking/mlruns/1/d0b1526fbb3c4161b2ae442c0f246bfe/artifacts/models_mlflow/model.xgb')

4514409

In [42]:
y_pred = xgboost_model.predict(valid)
y_pred[:10]

array([14.702353 ,  7.2431264, 13.131918 , 24.339914 ,  9.355686 ,
       17.190315 , 11.121852 ,  8.551083 ,  9.549852 , 19.934675 ],
      dtype=float32)

In [43]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

/home/michal/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
